### BERT 양쪽을 문맥을 이해
    - 빈칸채우기 

####
    - 데이터
    - Hugging Face Transformers Pipeline(간단한 api)    
    - 토크나이져 ( 텍스트 -> 토큰 ->숫자 ID)
    - 사전학습된 모델(BERT , GPT-2, DistilBERT 등)
    - 예측 결과(감성분석, 텍스트 생성, 문장 유사도)
    - 후처리 평가(확률 변환, 정확도 계산)

```
"I went to the [MASK] to buy some milk"
BERT 예측 : [MASK] = "store" ( 앞뒤 문맥 buy, milk를 보고 추론)
```

### 사전학습 방법
    - MLM(Masked Language Model) : 문장의 15% 단어를 MASK 처리한다음 예측
    - NSP(Next Stentece Prediction) : 두 문장이 연결되는지 판단
```
사전학습(대규모 텍스트)
[CLS] token 추가
특정 태스크 레이블로 학습(감성분석, QA 등)
```    

### DistilBERT
    - BERT 압축버전 : 속도는 2배 빠르다, 성능은 95% 유지함
    - 두꺼운 교과서(BERT)의 핵심만 추린 요약본(DistilBERT)
### 지식증류(Knowledge Distillation)
    - Teacher 모델(BERT) : 소프트레이블 생성(확률분포)
    - Student 모델(DistilBERT) : Teacher 출력을 모방
### GPT-2
    - 이전단어들을 보고 다음 단어를 예측    
    - 단방향 Attention:
    - Zero-shot Learning : 특정 태스트 학습 없이도 수행 가능
    - GPT 시리즈
        - GPT -1 : 117M 파라메터
        - GPT -2 : 1.5B 파라메터
        - GPT -3 : 175B 파라메터(Few-shot Learning)
        - GPT -4 : 멀티모달


In [1]:
import numpy as np
def softmax(logits):
    '''
    로짓을 확률 분포로 변환
    Args:
        logits : 모델의 출력 점수( [2.5,1.0,0.5])
    returns:
        확률 분포(합이 1인 배열)
    '''
    # 수치 안정성을 위해 최대값을 빼줌(오버플로우 방지)
    exp_logits = np.exp(logits - np.max(logits))
    return exp_logits / np.sum(exp_logits)
logits = np.array([2.5,1.0,0.5])    
probs = softmax(logits)
print(f'logits : {logits}')
print(f'softmax : {probs}')
print(f'softmax sum : {np.sum(probs)}')

logits : [2.5 1.  0.5]
softmax : [0.73612472 0.16425163 0.09962365]
softmax sum : 0.9999999999999999


### WordPiece   Word + Piece
    - 단어를 자주 등장하는 조각(piece) 단위로 잘라서 처리
    - 기존토크나이져 대비 --> 더 잘게 쪼개자. (Subword)
    - playing ----->  play + ##ing    ## 앞 조각에 붙는 서브워드 라는 의미
    - 득템   ---------> 득 + 템

WordPiece 토큰화 시뮬레이션

In [ ]:
%conda install transformers

In [11]:
from transformers import AutoTokenizer
# BERT 토크나이져 로드
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# 테스트 문장
sentence = 'I love natural language processing!'
# 토큰화
tokens = tokenizer.tokenize(sentence)
token_ids = tokenizer.encode(sentence)
# 디코딩
decoded = tokenizer.decode(token_ids)
print(f'원본문장 : {sentence}')
print(f'토큰목록 : {tokens}')
print(f'토큰ID : {token_ids}')
print(f'디코딩결과 : {decoded}')

print(f'CSL토큰 : {tokenizer.cls_token} -> {tokenizer.cls_token_id}')
print(f'SEP토큰 : {tokenizer.sep_token} -> {tokenizer.sep_token_id}')
print(f'PAD토큰 : {tokenizer.pad_token} -> {tokenizer.pad_token_id}')

원본문장 : I love natural language processing!
토큰목록 : ['i', 'love', 'natural', 'language', 'processing', '!']
토큰ID : [101, 1045, 2293, 3019, 2653, 6364, 999, 102]
디코딩결과 : [CLS] i love natural language processing! [SEP]
CSL토큰 : [CLS] -> 101
SEP토큰 : [SEP] -> 102
PAD토큰 : [PAD] -> 0


c:\Users\playdata2\miniconda3\envs\LLM311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERT로 문장 유사도 판단

In [ ]:
%pip install torch torchvision torchaudio

In [3]:
# AutoTokenizer  지정한 모델 이름에 맞게 토크나이져를 자동으로 불러오는 클래스
# AutoModelForSequenceClassification : 문장분류용 BERt모델을 자동으로 로드
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 모델 로드
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased-finetuned-mrpc')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased-finetuned-mrpc')
def check_similarity(sentence1, sentence2):
    '''
    두 문장의 의미적 유사도를 판단
    Returns:
        유사확률(0~1 사이 값)
    '''
    #1 토큰화
    inputs = tokenizer(sentence1,sentence2,return_tensors='pt')
    #2 모델추론 
    with torch.no_grad():
        logits = model(**inputs).logits
    #3 softmax로 확률 변환
        probs = torch.softmax(logits, dim=1)[0]  # 0은 배치 배치를 제거
    #4 결과 반환
    return{
        'not_similar' : probs[0].item(),
        'similar' : probs[1].item()
    }
# 테스트 케이스
test_cases = [
    ("The cat is on the mat", "A feline is sitting on a rug"),
    ("I love pizza", "Python is a programming language"),
    ("He runs fast", "She walks slowly")
]
for sent1,sent2 in test_cases:
    result = check_similarity(sent1,sent2)
    print(f'문장1 : {sent1}')
    print(f'문장2 : {sent2}')
    print(f"유사확률 : {result['similar']:.2f}\n")

c:\Users\playdata2\miniconda3\envs\LLM311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


문장1 : The cat is on the mat
문장2 : A feline is sitting on a rug
유사확률 : 0.90

문장1 : I love pizza
문장2 : Python is a programming language
유사확률 : 0.04

문장1 : He runs fast
문장2 : She walks slowly
유사확률 : 0.11



GPT-2
```
다음 단어 후보와 확률
The cat is on the 다음에 단어 후보 와 확률
mat             0.4
roof            0.25
bed             0.15
chair           0.10
floor           0.05

top-k  3  mat roof  bed
top-p  0.8

mat  0.40  -> 누적 0.40
roof 0.25  -> 누적 0.65
bed 0.15   -> 누적 0.80 
```

In [5]:
from transformers import GPT2LMHeadModel,GPT2Tokenizer
import torch
# 모델 로드
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')
def generate_text(prompt, max_length = 30):
    '''
    프롬프트 기반으로 텍스트 생성
    Args:
        prompt : 시작문장
        max_length : 최대 토큰수
    '''
    #1 입력 토큰화
    input_ids = tokenizer.encode(prompt,return_tensors='pt')
    #2 생성(다양한 전략)
    with torch.no_grad():  # 추론(평가)모드
        output = model.generate(
            input_ids,
            max_length = max_length,
            num_return_sequences = 1, # 생성할 문장 수
            temperature=0.8, # 창의성 조절
            top_k=50, # 샘플링전략  상위 k개의 단어만 선택
            top_p=0.95, # 누적확률 p 이상 단어만
            do_sample = True,  #확률적 셈플링
            pad_token_id = tokenizer.eos_token_id  # gpt2는 eos 토큰을 사용
        )

    #3 디코딩
    generated_txt = tokenizer.decode(output[0],skip_special_tokens=True)
    return generated_txt
prompts = [
    "Once upon a time",
    "In the year 2050,",
    "The secret to happiness is"
]
for prompt in prompts:
    result = generate_text(prompt)
    print(f'prompt : {prompt}')
    print(f'생성 : {result}\n')

c:\Users\playdata2\miniconda3\envs\LLM311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


prompt : Once upon a time
생성 : Once upon a time, there were seven or eight people who were doing it in their homes, and they said, "Hey, that's how I

prompt : In the year 2050,
생성 : In the year 2050, the global temperature will rise to 1.3 C above pre-industrial levels and reach more than 2 C above the pre-

prompt : The secret to happiness is
생성 : The secret to happiness is to feel it. If it's not, nothing is.

* * *

A year ago, at least

